### Audio track loading and visualization

In [ ]:
# Define audio file
audio_file = "..//samples//piano_oneshot.mp3"

### Track characterization

Here, 

In [ ]:
import numpy as np
from pydub import AudioSegment

song = AudioSegment.from_mp3(audio_file)

# Parse track into channels
channel_sounds = song.split_to_mono()
signals = [s.get_array_of_samples() for s in channel_sounds]
fp_arr = np.array(signals).T.astype(np.float32)
fp_arr /= np.iinfo(signals[0].typecode).max  # Shape: (464256,2)

# Extract track characteristics
num_samp = fp_arr.shape[0]
duration = song.duration_seconds
song_time = np.linspace(start=0, stop=duration, num=num_samp)

### Plot each isolated audio channel

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("dark_background")
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 4))
plt.suptitle("Audio Plot")
ax1.plot(song_time, fp_arr[:, 0], color="green", linewidth=0.5)
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Signal wave")
ax1.set_xlim(0, duration)
ax2.plot(song_time, fp_arr[:, 0], color="orange", linewidth=0.5)
ax2.set_xlim(0, duration)
ax2.set_xlabel("Time [s]")
plt.tight_layout()
plt.show()

### Mess with one side of the audio

### Substitute one audio side for another track

### Plot channels in real-time as they play

In [ ]:
"""
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1) 

for i in range(20):
    x = np.arange(0, i, 0.1)
    y = np.sin(x)
    
    ax.set_xlim(0, i)
    
    ax.cla()
    ax.plot(x, y)
    display(fig)
    
    clear_output(wait = True)
    plt.pause(0.5)
"""